In [55]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import time, datetime
import requests
import io
import re
import json
import validators

## Stripping data from the old notion page

In [39]:
# initialize the web driver
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)

# open the notion page
driver.get('https://www.notion.so/A-Guide-to-L-A-Coffee-Tea-Shops-3042e354ce5745d1940bed1a272a53a5')

# load the whole table
driver.find_element("xpath", "//*[contains(text(), 'Load more')]").click()

# notion-table-view-cell elements are containers for each table entry
cell_elements = driver.find_elements("class name", "notion-table-view-cell")

In [5]:
def getCellText(cell_element):
    children = cell_element.find_elements("xpath", ".//*")
    for child in children:
        if child.text:
            return child.text
    return None

def getProgressBarData(cell_element):
    children = cell_element.find_elements("xpath", ".//*")
    for child in children:
        if child.get_attribute("role") == "progressbar":
            return child.get_attribute("aria-valuenow")
    return None

def getCellRowIndex(cell_element):
    return int(cell_element.get_attribute("data-row-index"))

def getCellColIndex(cell_element):
    return int(cell_element.get_attribute("data-col-index"))

In [12]:
# parse the html elements into a python dict

headers = [
    "score",
    "name",
    "neighborhood",
    "ambiance",
    "workability",
    "drinks",
    "outlets",
    "tags"
]

cafeObjects = []

for element in cell_elements:
    col_index = getCellColIndex(element)
    row_index = getCellRowIndex(element)
    
    # allocate a new object for every table row
    if col_index == 0:
        object = {}
        object["index"] = row_index
        cafeObjects.append(object)
        
    if col_index >= len(headers):
        continue
    
    # add a pair to the dict
    header = headers[col_index]
    value = None
    match header:
        case "score":
            value = getCellText(element)
        case "name":
            value = getCellText(element)
        case "neighborhood":
            value = getCellText(element)
        case "ambiance":
            value = getProgressBarData(element)
        case "workability":
            value = getProgressBarData(element)
        case "drinks":
            value = getProgressBarData(element)
        case "outlets":
            value = getProgressBarData(element)
        case "tags":
            value = getCellText(element)
            if value == None:
                value = ""
        case _:
            value = None
    object[header] = value
    
# parse the series of boolean values stored in the "tags" property
for cafe in cafeObjects:
    if cafe["tags"] == None:
        continue
    cafe["open_late"] = cafe["tags"].find("Open Late") != -1
    cafe["has_food"] = cafe["tags"].find("Has Food") != -1
    cafe["hidden_gem"] = cafe["tags"].find("Hidden Gem") != -1
    cafe["outdoor_area"] = cafe["tags"].find("Outdoor Area") != -1
    cafe["closes_early"] = cafe["tags"].find("Closes Early") != -1
    cafe["high_prices"] = cafe["tags"].find("High Prices") != -1
    cafe["wifi_issues"] = cafe["tags"].find("Wifi-Issues") != -1
    cafe["difficult_seating"] = cafe["tags"].find("Difficult Seating") != -1
    
for cafe in cafeObjects:
    name = cafe["name"]
    subname = ""
    start = name.find("[")
    if start != -1:
        end = name.find("]")
        subname = name[start:end+1]
        cafe["name"] = name[:start]
    cafe["subname"] = subname   
    
# we also get rows belonging to the "cafes to visit" table, but they can be 
# dropped because they don't parse properly and contain values equal to None
notiondf = pd.DataFrame(cafeObjects).dropna()

notiondf


,index,score,name,neighborhood,ambiance,workability,drinks,outlets,tags,open_late,has_food,hidden_gem,outdoor_area,closes_early,high_prices,wifi_issues,difficult_seating,subname
0,0,14.5,Steep,Echo Park/Silver Lake/Chinatown,100,100,80,50,Has Food\nOpen Late\nOutdoor Area\nHidden Gem,True,True,True,True,False,False,False,False,
1,1,14,Coffee MCO,Koreatown/Mid-City,100,80,80,100,Has Food\nOutdoor Area,False,True,False,True,False,False,False,False,
2,2,14,3THYME Coffee,Koreatown/Mid-City,80,80,100,100,Wi-Fi Issues\nOutdoor Area\nHidden Gem,False,False,True,True,False,False,False,False,[Koreatown]
3,3,13.5,Maru Coffee,Arts District/Little Tokyo,100,60,100,100,Difficult Seating,False,False,False,False,False,False,False,True,[Arts District]
4,4,13,Alchemist Coffee Project: The Pearl,Koreatown/Mid-City,80,100,60,100,High Prices,False,False,False,False,False,True,False,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,10,Intelligentsia,Los Feliz/Hollywood,60,40,80,0,,False,False,False,False,False,False,False,False,[Hollywood]
63,63,9,Verve Coffee Roasters,WeHo/Melrose/Westwood/Venice,80,60,40,0,,False,False,False,False,False,False,False,False,[WeHo]
64,64,9,Blue Bottle Coffee,Arts District/Little Tokyo,60,40,80,0,Difficult Seating,False,False,False,False,False,False,False,True,[Arts District]
65,65,9,Ministry of Coffee,USC/South Central,60,80,20,100,High Prices,False,False,False,False,False,True,False,False,


In [13]:
# add and remove rows to match desired format

notiondf.drop(["tags"], axis=1)

notiondf["address"] = ""
notiondf["color_code"] = ""
notiondf["hours"] = ""
notiondf["latitude"] = ""
notiondf["longitude"] = ""
notiondf["image"] = ""
notiondf["yelp"] = ""
notiondf["google_maps"] = ""
notiondf["study_work"] = ""
notiondf["is_aesthetic"] = ""
notiondf["visible"] = True
notiondf["is_selected"] = False
notiondf["is_hovered"] = False
notiondf["n_latitude"] = ""
notiondf["n_longitude"] = ""
notiondf["n_zoom"] = ""

notiondf = notiondf.loc[:, [
    "index",
    "name",
    "subname",
    "neighborhood",
    "color_code",
    "address",
    "hours",
    "latitude",
    "longitude",
    "yelp",
    "google_maps",
    "score",
    "ambiance",
    "workability",
    "drinks",
    "outlets",
    "study_work",
    "has_food",
    "hidden_gem",
    "open_late",
    "is_aesthetic",
    "outdoor_area",
    "closes_early",
    "high_prices",
    "wifi_issues",
    "difficult_seating",
    "visible",
    "is_selected",
    "is_hovered",
    "n_latitude",
    "n_longitude",
    "n_zoom"]]

In [14]:
neighborhoods = {
    "Echo Park/Silver Lake/Chinatown": {
        "color_code": "#F6C25C",
        "n_latitude": 34.07769323,
        "n_longitude": -118.2654183,
        "n_zoom": 13
    },
    "Koreatown/Mid-City": {
        "color_code": "#867BC0",
        "n_latitude": 34.06254827,
        "n_longitude": -118.3090395,
        "n_zoom": 13
    },
    "Arts District/Little Tokyo": {
        "color_code": "#5FC5F9",
        "n_latitude": 34.03944538,
        "n_longitude": -118.2357442,
        "n_zoom": 13
    },
    "Downtown": {
        "color_code": "#3683C2",
        "n_latitude": 34.05378215,
        "n_longitude": -118.2531122,
        "n_zoom": 13
    },
    "WeHo/Melrose/Beverly Hills": {
        "color_code": "#F2729F",
        "n_latitude": 34.07642731,
        "n_longitude": -118.3674769,
        "n_zoom": 13
    },
    "WeHo/Melrose/Westwood/Venice": {
        "new_name": "WeHo/Melrose/Beverly Hills",
        "color_code": "#F2729F",
        "n_latitude": 34.07642731,
        "n_longitude": -118.3674769,
        "n_zoom": 13
    },
    "Culver City/Mar Vista": {
        "color_code": "#74B78C",
        "n_latitude": 34.01161969,
        "n_longitude": -118.4039303,
        "n_zoom": 13
    },
    "USC/South Central": {
        "color_code": "#FF6961",
        "n_latitude": 34.02902002,
        "n_longitude": -118.3001098,
        "n_zoom": 13
    },
    "Highland Park/Eagle Rock": {
        "color_code": "#FF0000",
        "n_latitude": 0,
        "n_longitude": 0,
        "n_zoom": 13
    },
    "Los Feliz/Hollywood": {
        "color_code": "#FF0000",
        "n_latitude": 0,
        "n_longitude": 0,
        "n_zoom": 13
    },
}

def evaluateNewNeighborhoodName(neighborhood):
    if "new_name" in neighborhoods[neighborhood]:
        return neighborhoods[neighborhood]["new_name"]
    else:
        return neighborhood

notiondf["color_code"] = pd.Series([neighborhoods[neighborhood]["color_code"] for neighborhood in notiondf["neighborhood"]])
notiondf["n_latitude"] = pd.Series([neighborhoods[neighborhood]["n_latitude"] for neighborhood in notiondf["neighborhood"]])
notiondf["n_longitude"] = pd.Series([neighborhoods[neighborhood]["n_longitude"] for neighborhood in notiondf["neighborhood"]])
notiondf["n_zoom"] = pd.Series([neighborhoods[neighborhood]["n_zoom"] for neighborhood in notiondf["neighborhood"]])
notiondf["neighborhood"] = pd.Series([evaluateNewNeighborhoodName(neighborhood) for neighborhood in notiondf["neighborhood"]])

notiondf

,index,name,subname,neighborhood,color_code,address,hours,latitude,longitude,yelp,...,closes_early,high_prices,wifi_issues,difficult_seating,visible,is_selected,is_hovered,n_latitude,n_longitude,n_zoom
0,0,Steep,,Echo Park/Silver Lake/Chinatown,#F6C25C,,,,,,...,False,False,False,False,True,False,False,34.077693,-118.265418,13
1,1,Coffee MCO,,Koreatown/Mid-City,#867BC0,,,,,,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
2,2,3THYME Coffee,[Koreatown],Koreatown/Mid-City,#867BC0,,,,,,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
3,3,Maru Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,,,,,,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
4,4,Alchemist Coffee Project: The Pearl,,Koreatown/Mid-City,#867BC0,,,,,,...,False,True,False,False,True,False,False,34.062548,-118.309039,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,Intelligentsia,[Hollywood],Los Feliz/Hollywood,#FF0000,,,,,,...,False,False,False,False,True,False,False,0.000000,0.000000,13
63,63,Verve Coffee Roasters,[WeHo],WeHo/Melrose/Beverly Hills,#F2729F,,,,,,...,False,False,False,False,True,False,False,34.076427,-118.367477,13
64,64,Blue Bottle Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,,,,,,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
65,65,Ministry of Coffee,,USC/South Central,#FF6961,,,,,,...,False,True,False,False,True,False,False,34.029020,-118.300110,13


In [94]:
notiondf.to_csv("./notion.csv", index=False)

## Filling in more data

In [89]:
notiondf = pd.read_csv("./notion.csv", keep_default_na=False)
notiondf

,index,name,subname,neighborhood,color_code,address,hours,latitude,longitude,yelp,...,closes_early,high_prices,wifi_issues,difficult_seating,visible,is_selected,is_hovered,n_latitude,n_longitude,n_zoom
0,0,Steep,,Echo Park/Silver Lake/Chinatown,#F6C25C,"970 N Broadway Ste 112 Los Angeles, CA 90012","[{""open"": 1100, ""close"": 2300}, {""open"": 0, ""c...",,,https://www.yelp.com/biz/steep-los-angeles-2,...,False,False,False,False,True,False,False,34.077693,-118.265418,13
1,1,Coffee MCO,,Koreatown/Mid-City,#867BC0,"2580 W Olympic Blvd Los Angeles, CA 90006","[{""open"": 800, ""close"": 1900}, {""open"": 800, ""...",,,https://www.yelp.com/biz/coffee-mco-los-angeles,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
2,2,3THYME Coffee,[Koreatown],Koreatown/Mid-City,#867BC0,"600 S Harvard Blvd Ste 100 Los Angeles, CA 90005","[{""open"": 800, ""close"": 2000}, {""open"": 800, ""...",,,https://www.yelp.com/biz/3thyme-coffee-los-ang...,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
3,3,Maru Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,"1019 S Santa Fe Ave Los Angeles, CA 90021","[{""open"": 800, ""close"": 1700}, {""open"": 730, ""...",,,https://www.yelp.com/biz/maru-coffee-los-angel...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
4,4,Alchemist Coffee Project: The Pearl,,Koreatown/Mid-City,#867BC0,"698 S Vermont Ave Ste 103 Los Angeles, CA 90005","[{""open"": 730, ""close"": 2000}, {""open"": 700, ""...",,,https://www.yelp.com/biz/alchemist-coffee-proj...,...,False,True,False,False,True,False,False,34.062548,-118.309039,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,Intelligentsia,[Hollywood],Los Feliz/Hollywood,#FF0000,,,,,https://www.yelp.com/biz/intelligentsia-coffee...,...,False,False,False,False,True,False,False,0.000000,0.000000,13
63,63,Verve Coffee Roasters,[WeHo],WeHo/Melrose/Beverly Hills,#F2729F,,,,,https://www.yelp.com/biz/verve-coffee-roasters...,...,False,False,False,False,True,False,False,34.076427,-118.367477,13
64,64,Blue Bottle Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,,,,,https://www.yelp.com/biz/blue-bottle-coffee-ar...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
65,65,Ministry of Coffee,,USC/South Central,#FF6961,,,,,https://www.yelp.com/biz/ministry-of-coffee-lo...,...,False,True,False,False,True,False,False,34.029020,-118.300110,13


In [90]:
def getHoursFromYelp(driver):
    hour_elements = driver.find_elements("class name", "y-css-29kerx")
    hours = list(filter(None, [element.text for element in hour_elements]))
    hours = hours[:7]
    hours.insert(0, hours[-1])
    hours.pop()

    bad_strings = [
        'Mon\n',
        'Tue\n',
        'Wed\n',
        'Thu\n',
        'Fri\n',
        'Sat\n',
        'Sun\n',
        '\nClosed now',
        'Closed',
        '\nOpen now',
        ' '
    ]

    for bad_string in bad_strings:
        hours = [element.replace(bad_string, '') for element in hours]
        
    hours = [element.replace('-', ',') for element in hours]
    
    hours_object = []

    for element in hours:
        obj = {}
        if element != '':
            split = element.partition(',')
            open = datetime.strptime(split[0], '%I:%M%p').time()
            close = datetime.strptime(split[2], '%I:%M%p').time()
            obj["open"] = open.hour*100 + open.minute
            obj["close"] = close.hour*100 + close.minute
        else:
            obj["open"] = 0
            obj["close"] = 0    
        hours_object.append(obj)
        
    return json.dumps(hours_object)

def getAddressFromYelp(driver):
    badStrings = [
        "Sponsored",
        "No Reservations",
        "No Delivery",
        "No Takeout"
    ]
    elements = driver.find_elements("class name", "y-css-dg8xxd")
    for element in elements:
        if element.text not in badStrings:
            return element.text
    return None

In [91]:
hoursList = []
addressList = []

for yelp in notiondf["yelp"]:
    hours = ""
    address = ""
    if validators.url(yelp):
        driver.get(yelp)
        try:
            hours = getHoursFromYelp(driver)
            address = getAddressFromYelp(driver)
        except:
            hours = "ERROR"
            address = "ERROR"
    hoursList.append(hours)
    addressList.append(address)
    

In [92]:
notiondf["hours"] = pd.Series(hoursList)
notiondf["address"] = pd.Series(addressList)
notiondf.where(notiondf.notnull(), "")

,index,name,subname,neighborhood,color_code,address,hours,latitude,longitude,yelp,...,closes_early,high_prices,wifi_issues,difficult_seating,visible,is_selected,is_hovered,n_latitude,n_longitude,n_zoom
0,0,Steep,,Echo Park/Silver Lake/Chinatown,#F6C25C,"970 N Broadway Ste 112 Los Angeles, CA 90012","[{""open"": 1100, ""close"": 2300}, {""open"": 0, ""c...",,,https://www.yelp.com/biz/steep-los-angeles-2,...,False,False,False,False,True,False,False,34.077693,-118.265418,13
1,1,Coffee MCO,,Koreatown/Mid-City,#867BC0,"2580 W Olympic Blvd Los Angeles, CA 90006","[{""open"": 800, ""close"": 1900}, {""open"": 800, ""...",,,https://www.yelp.com/biz/coffee-mco-los-angeles,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
2,2,3THYME Coffee,[Koreatown],Koreatown/Mid-City,#867BC0,"600 S Harvard Blvd Ste 100 Los Angeles, CA 90005","[{""open"": 800, ""close"": 2000}, {""open"": 800, ""...",,,https://www.yelp.com/biz/3thyme-coffee-los-ang...,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
3,3,Maru Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,"1019 S Santa Fe Ave Los Angeles, CA 90021","[{""open"": 800, ""close"": 1700}, {""open"": 730, ""...",,,https://www.yelp.com/biz/maru-coffee-los-angel...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
4,4,Alchemist Coffee Project: The Pearl,,Koreatown/Mid-City,#867BC0,"687 S Hobart Blvd Unit C Los Angeles, CA 90005","[{""open"": 730, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/alchemist-coffee-proj...,...,False,True,False,False,True,False,False,34.062548,-118.309039,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,Intelligentsia,[Hollywood],Los Feliz/Hollywood,#FF0000,"6401 Hollywood Blvd Los Angeles, CA 90028","[{""open"": 700, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/intelligentsia-coffee...,...,False,False,False,False,True,False,False,0.000000,0.000000,13
63,63,Verve Coffee Roasters,[WeHo],WeHo/Melrose/Beverly Hills,#F2729F,"8925 Melrose Ave West Hollywood, CA 90069","[{""open"": 700, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/verve-coffee-roasters...,...,False,False,False,False,True,False,False,34.076427,-118.367477,13
64,64,Blue Bottle Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,"582 Mateo St Los Angeles, CA 90013","[{""open"": 700, ""close"": 1800}, {""open"": 630, ""...",,,https://www.yelp.com/biz/blue-bottle-coffee-ar...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
65,65,Ministry of Coffee,,USC/South Central,#FF6961,"844 W 32nd St Los Angeles, CA 90007","[{""open"": 0, ""close"": 0}, {""open"": 730, ""close...",,,https://www.yelp.com/biz/ministry-of-coffee-lo...,...,False,True,False,False,True,False,False,34.029020,-118.300110,13


In [93]:
notiondf

,index,name,subname,neighborhood,color_code,address,hours,latitude,longitude,yelp,...,closes_early,high_prices,wifi_issues,difficult_seating,visible,is_selected,is_hovered,n_latitude,n_longitude,n_zoom
0,0,Steep,,Echo Park/Silver Lake/Chinatown,#F6C25C,"970 N Broadway Ste 112 Los Angeles, CA 90012","[{""open"": 1100, ""close"": 2300}, {""open"": 0, ""c...",,,https://www.yelp.com/biz/steep-los-angeles-2,...,False,False,False,False,True,False,False,34.077693,-118.265418,13
1,1,Coffee MCO,,Koreatown/Mid-City,#867BC0,"2580 W Olympic Blvd Los Angeles, CA 90006","[{""open"": 800, ""close"": 1900}, {""open"": 800, ""...",,,https://www.yelp.com/biz/coffee-mco-los-angeles,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
2,2,3THYME Coffee,[Koreatown],Koreatown/Mid-City,#867BC0,"600 S Harvard Blvd Ste 100 Los Angeles, CA 90005","[{""open"": 800, ""close"": 2000}, {""open"": 800, ""...",,,https://www.yelp.com/biz/3thyme-coffee-los-ang...,...,False,False,False,False,True,False,False,34.062548,-118.309039,13
3,3,Maru Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,"1019 S Santa Fe Ave Los Angeles, CA 90021","[{""open"": 800, ""close"": 1700}, {""open"": 730, ""...",,,https://www.yelp.com/biz/maru-coffee-los-angel...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
4,4,Alchemist Coffee Project: The Pearl,,Koreatown/Mid-City,#867BC0,"687 S Hobart Blvd Unit C Los Angeles, CA 90005","[{""open"": 730, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/alchemist-coffee-proj...,...,False,True,False,False,True,False,False,34.062548,-118.309039,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,Intelligentsia,[Hollywood],Los Feliz/Hollywood,#FF0000,"6401 Hollywood Blvd Los Angeles, CA 90028","[{""open"": 700, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/intelligentsia-coffee...,...,False,False,False,False,True,False,False,0.000000,0.000000,13
63,63,Verve Coffee Roasters,[WeHo],WeHo/Melrose/Beverly Hills,#F2729F,"8925 Melrose Ave West Hollywood, CA 90069","[{""open"": 700, ""close"": 1800}, {""open"": 700, ""...",,,https://www.yelp.com/biz/verve-coffee-roasters...,...,False,False,False,False,True,False,False,34.076427,-118.367477,13
64,64,Blue Bottle Coffee,[Arts District],Arts District/Little Tokyo,#5FC5F9,"582 Mateo St Los Angeles, CA 90013","[{""open"": 700, ""close"": 1800}, {""open"": 630, ""...",,,https://www.yelp.com/biz/blue-bottle-coffee-ar...,...,False,False,False,True,True,False,False,34.039445,-118.235744,13
65,65,Ministry of Coffee,,USC/South Central,#FF6961,"844 W 32nd St Los Angeles, CA 90007","[{""open"": 0, ""close"": 0}, {""open"": 730, ""close...",,,https://www.yelp.com/biz/ministry-of-coffee-lo...,...,False,True,False,False,True,False,False,34.029020,-118.300110,13
